# AlloyDB Semantic Search: Team USA Athletes

In this notebook, you'll set up **AlloyDB** as the operational search layer for the Team USA analytics platform.

While BigQuery handles analytical queries (aggregations, clustering, trends), AlloyDB provides the **fast, real-time similarity search** that an AI agent needs — answering questions like *"which athletes had careers most similar to Simone Biles?"* in milliseconds.

**What you'll do:**
1. Create the `team_usa` database on your AlloyDB cluster
2. Load 12,000+ athletes with pre-computed vector embeddings
3. Build a ScaNN index for Google-scale similarity search
4. Run semantic queries that find athletes by meaning, not keywords
5. Search by natural language description — the same pattern the agent will use
6. Load 25,000+ individual event results linked to athletes

---
## Step 1: Configure Your Environment

Update the fields below with your lab-specific values. The cluster and instance IDs should match your Terraform deployment from Task 1.

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
REGION = "YOUR_LAB_REGION"  # @param {type:"string"}
USER_EMAIL = "YOUR_LAB_USER"  # @param {type:"string"}

# These should match your Terraform deployment
CLUSTER_ID = "team-usa-cluster"  # @param {type:"string"}
INSTANCE_ID = "team-usa-primary"  # @param {type:"string"}

# Derived values
DATABASE_NAME = "team_usa"
INSTANCE_URI = f"projects/{PROJECT_ID}/locations/{REGION}/clusters/{CLUSTER_ID}/instances/{INSTANCE_ID}"

print("📋 Configuration:")
print(f"   Project:  {PROJECT_ID}")
print(f"   Region:   {REGION}")
print(f"   Cluster:  {CLUSTER_ID}")
print(f"   Instance: {INSTANCE_ID}")
print(f"   Database: {DATABASE_NAME}")
print(f"   User:     {USER_EMAIL}")
print(f"\n   Instance URI: {INSTANCE_URI}")

---
## Step 2: Install Dependencies & Connect to AlloyDB

The **AlloyDB Python Connector** handles secure IAM authentication — no passwords needed. Your Google Cloud identity *is* your database identity.

In [ ]:
%pip install --quiet google-cloud-alloydb-connector[pg8000] sqlalchemy pandas

In [ ]:
from google.cloud.alloydb.connector import Connector, IPTypes
import pg8000
import sqlalchemy
from sqlalchemy import text

connector = Connector()

def get_connection(database="postgres"):
    """Create a connection to AlloyDB."""
    conn = connector.connect(
        INSTANCE_URI,
        "pg8000",
        user=USER_EMAIL,
        db=database,
        enable_iam_auth=True,
        ip_type=IPTypes.PUBLIC,
    )
    return conn

# Test the connection against the default postgres database
conn = get_connection("postgres")
cursor = conn.cursor()
cursor.execute("SELECT version()")
version = cursor.fetchone()[0]
conn.close()

print(f"✅ Connected to AlloyDB!")
print(f"🔐 Authenticated as: {USER_EMAIL}")
print(f"📊 {version[:70]}...")

---
## Step 3: Create the Database

The AlloyDB cluster has been running since Task 1, but only with the default `postgres` database. Let's create a dedicated `team_usa` database for our athlete data.

In [ ]:
# CREATE DATABASE can't run inside a transaction, so we use autocommit
conn = get_connection("postgres")
conn.autocommit = True
cursor = conn.cursor()

cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (DATABASE_NAME,))
if cursor.fetchone() is None:
    cursor.execute(f"CREATE DATABASE {DATABASE_NAME}")
    print(f"✅ Database '{DATABASE_NAME}' created!")
else:
    print(f"ℹ️  Database '{DATABASE_NAME}' already exists.")

conn.close()

Now connect to the new `team_usa` database. This is the connection you'll use for everything that follows.

In [ ]:
# Create a SQLAlchemy engine connected to team_usa
engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=lambda: get_connection(DATABASE_NAME),
)

# Verify the connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT current_database(), current_user"))
    db, user = result.fetchone()
    print(f"✅ Connected to database: {db}")
    print(f"   Authenticated as: {user}")

---
## Step 4: Enable Extensions

Two extensions give AlloyDB its vector search capabilities:

- **vector** — Adds the `VECTOR` data type for storing embeddings
- **alloydb_scann** — Google's [ScaNN](https://github.com/google-research/google-research/tree/master/scann) (Scalable Nearest Neighbors) algorithm for high-performance vector similarity search
- **pgaudit** — Provides detailed logging of database operations for security and compliance

In [ ]:
with engine.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector"))
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS alloydb_scann"))
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS pgaudit"))

    conn.commit()

    # Verify
    result = conn.execute(text(
        "SELECT extname, extversion FROM pg_extension "
        "WHERE extname IN ('vector', 'alloydb_scann', 'pgaudit') ORDER BY extname"
    ))
    for row in result:
        print(f"✅ {row[0]} v{row[1]}")

---
## Step 5: Create the Athletes Table

The table includes standard columns for display and filtering, plus a `VECTOR(3072)` column for the pre-computed embeddings.

We're loading 13 of the 29 columns from the full dataset — the ones needed for search results display, filtering, and vector search. Columns like birth place, height/weight, and sparse Paralympic personal details are left in BigQuery where they're available if needed.

In [ ]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS athletes"))
    conn.execute(text("""
        CREATE TABLE athletes (
          athlete_id          VARCHAR(36)  PRIMARY KEY,
          name                VARCHAR      NOT NULL,
          gender              VARCHAR,
          games_type          VARCHAR      NOT NULL,
          games_season        VARCHAR      NOT NULL,
          primary_sport       VARCHAR      NOT NULL,
          classification_code VARCHAR,
          total_medals        INTEGER      DEFAULT 0,
          gold_count          INTEGER      DEFAULT 0,
          silver_count        INTEGER      DEFAULT 0,
          bronze_count        INTEGER      DEFAULT 0,
          games_count         INTEGER      DEFAULT 0,
          first_games_year    INTEGER,
          last_games_year     INTEGER,
          profile_summary     TEXT,
          embedding           VECTOR(3072)
      )
    """))
    conn.commit()

print("✅ Athletes table created!")
print("   Columns: athlete_id, name, gender, games_type, primary_sport,")
print("            classification_code, total_medals, gold_count, games_count,")
print("            first_games_year, last_games_year, profile_summary, embedding")

---
## Step 6: Load Athlete Data

We'll load data using AlloyDB's **GCS import** — a server-side bulk load that's significantly faster than inserting rows through a client connection. The process:

1. Download the source CSV and select the columns we need
2. Stage the prepared data in a Cloud Storage bucket
3. Import directly into AlloyDB via server-side load

### Step 6.1: Download and prepare the data

In [ ]:
import pandas as pd

# Download the full athletes CSV from the lab's GCS bucket
print("⏳ Downloading athlete data from Cloud Storage...")
df = pd.read_csv("gs://class-demo/team-usa/final/team_usa_athletes.csv")
print(f"   Downloaded {len(df):,} athletes with {len(df.columns)} columns")

# Select the 16 columns we need, in table column order
columns = [
    'athlete_id', 'name', 'gender', 'games_type', 'games_season',
    'primary_sport', 'classification_code', 'total_medals', 'gold_count',
    'silver_count', 'bronze_count', 'games_count', 'first_games_year',
    'last_games_year', 'profile_summary', 'embedding'
]
df_alloydb = df[columns].copy()

# Fix pandas float-promotion for integer columns
int_columns = ['total_medals', 'gold_count', 'silver_count', 'bronze_count',
               'games_count', 'first_games_year', 'last_games_year']
for col in int_columns:
    df_alloydb[col] = df_alloydb[col].fillna(0).astype(int)

# Save without headers — the GCS import maps columns positionally
local_path = "/tmp/athletes_for_alloydb.csv"
df_alloydb.to_csv(local_path, index=False, header=False)

print(f"✅ Prepared {len(df_alloydb):,} athletes ({len(columns)} columns) for import")
print(f"   Saved to {local_path}")

### Step 6.2: Stage data to Cloud Storage

AlloyDB's GCS import reads directly from Cloud Storage, so we need to stage our prepared file in a bucket that the AlloyDB service agent can access.

In [ ]:
# Get the project number (needed to identify the AlloyDB service agent)
project_number = !gcloud projects describe {PROJECT_ID} --format="value(projectNumber)"
project_number = project_number[0].strip()
service_agent = f"service-{project_number}@gcp-sa-alloydb.iam.gserviceaccount.com"

STAGING_BUCKET = f"{PROJECT_ID}-team-usa-staging"

# Create the staging bucket
!gcloud storage buckets create gs://{STAGING_BUCKET} --location={REGION} 2>/dev/null || true

# Grant AlloyDB service agent read access
!gcloud storage buckets add-iam-policy-binding gs://{STAGING_BUCKET} \
    --member="serviceAccount:{service_agent}" \
    --role="roles/storage.objectViewer" --quiet

# Upload the prepared CSV
!gcloud storage cp {local_path} gs://{STAGING_BUCKET}/athletes_for_alloydb.csv --quiet

print(f"\n✅ Data staged to gs://{STAGING_BUCKET}/athletes_for_alloydb.csv")
print(f"   AlloyDB service agent ({service_agent[:40]}...) granted read access")

### Step 6.3: Import into AlloyDB

This triggers a **server-side import** — AlloyDB reads the CSV directly from Cloud Storage. Much faster than sending rows through a client connection.

In [ ]:
print("⏳ Importing athletes into AlloyDB (this may take 1-3 minutes)...")

!gcloud alloydb clusters import {CLUSTER_ID} \
    --project={PROJECT_ID} \
    --region={REGION} \
    --database={DATABASE_NAME} \
    --user={USER_EMAIL} \
    --csv \
    --table=athletes \
    --gcs-uri=gs://{STAGING_BUCKET}/athletes_for_alloydb.csv \
    --columns=athlete_id,name,gender,games_type,games_season,primary_sport,classification_code,total_medals,gold_count,silver_count,bronze_count,games_count,first_games_year,last_games_year,profile_summary,embedding

print("\n✅ Import complete!")

---
## Step 7: Verify the Data

Let's confirm everything loaded correctly.

In [ ]:
with engine.connect() as conn:
    # Total count
    result = conn.execute(text("SELECT COUNT(*) FROM athletes"))
    total = result.scalar()

    # Embedding count
    result = conn.execute(text("SELECT COUNT(*) FROM athletes WHERE embedding IS NOT NULL"))
    with_embeddings = result.scalar()

    # Games type breakdown
    result = conn.execute(text(
        "SELECT games_type, COUNT(*) FROM athletes GROUP BY games_type ORDER BY games_type"
    ))
    breakdown = result.fetchall()

    print(f"✅ Athletes loaded: {total:,}")
    print(f"   With embeddings: {with_embeddings:,}")
    for games_type, count in breakdown:
        print(f"   {games_type}: {count:,}")

    # Sample some athletes
    print("\n📋 Sample athletes:")
    result = conn.execute(text("""
        SELECT name, primary_sport, games_type,
               gold_count, silver_count, bronze_count, total_medals,
               first_games_year || '-' || last_games_year AS career
        FROM athletes
        WHERE total_medals > 5
        ORDER BY total_medals DESC
        LIMIT 8
    """))
    print(f"   {'Name':<30s} {'Sport':<25s} {'Type':<12s} {'🥇':>3s} {'🥈':>3s} {'🥉':>3s} {'Tot':>4s} {'Career'}")
    print(f"   {'-'*30} {'-'*25} {'-'*12} {'-'*3} {'-'*3} {'-'*3} {'-'*4} {'-'*9}")
    for row in result:
        print(f"   {row[0]:<30s} {row[1]:<25s} {row[2]:<12s} {row[3]:>3d} {row[4]:>3d} {row[5]:>3d} {row[6]:>4d} {row[7]}")

---
## Step 8: Create the ScaNN Index

Without an index, every similarity search compares your query against all 12,222 embeddings — a brute-force scan. The **ScaNN** index pre-organizes vectors into partitions so queries search only the most promising candidates.

- **cosine** distance: Measures the angle between vectors. Ideal for text embeddings.
- **num_leaves = 50**: Partitions vectors into 50 clusters for search.
- **quantizer = 'sq8'**: Compresses dimensions to 8-bit, reducing memory while preserving accuracy.

In [ ]:
import time

with engine.connect() as conn:
    print("⏳ Creating ScaNN index...")
    start = time.time()
    conn.execute(text("""
        CREATE INDEX athletes_embedding_idx
        ON athletes USING scann (embedding cosine)
        WITH (num_leaves = 50, quantizer = 'sq8')
    """))
    conn.commit()
    elapsed = time.time() - start
    print(f"✅ ScaNN index created in {elapsed:.1f}s")

    # Verify the index exists
    result = conn.execute(text("""
        SELECT indexname, indexdef
        FROM pg_indexes
        WHERE tablename = 'athletes' AND indexname = 'athletes_embedding_idx'
    """))
    idx = result.fetchone()
    if idx:
        print(f"   Index: {idx[0]}")

---
## Step 9: Find Similar Athletes

Now for the payoff. This query looks up a specific athlete's embedding and finds the nearest neighbors in vector space — athletes whose career "fingerprints" are closest in 3,072-dimensional space.

The `<=>` operator computes **cosine distance** between vectors (lower = more similar). We convert to a similarity score (higher = more similar) with `1 - distance`.

### Simone Biles — Dominant multi-Games gymnast

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            a2.name,
            a2.primary_sport,
            a2.games_type,
            a2.classification_code,
            a2.gold_count,
            a2.silver_count,
            a2.bronze_count,
            a2.total_medals,
            a2.first_games_year || '-' || a2.last_games_year AS career,
            ROUND((1 - (a1.embedding <=> a2.embedding))::numeric, 3) AS similarity
        FROM athletes a1
        CROSS JOIN athletes a2
        WHERE a1.name like '%Simone%Biles%'
          AND a2.athlete_id != a1.athlete_id
          AND a2.embedding IS NOT NULL
        ORDER BY a1.embedding <=> a2.embedding
        LIMIT 10
    """))

    print("🔍 Athletes most similar to Simone Biles:\n")
    print(f"   {'Name':<30s} {'Sport':<22s} {'Type':<10s} {'🥇':>3s} {'🥈':>3s} {'🥉':>3s} {'Tot':>4s} {'Career':<10s} {'Sim':>5s}")
    print(f"   {'-'*30} {'-'*22} {'-'*10} {'-'*3} {'-'*3} {'-'*3} {'-'*4} {'-'*10} {'-'*5}")
    for row in result:
        print(f"   {row[0]:<30s} {row[1]:<22s} {row[2]:<10s} {row[4]:>3d} {row[5]:>3d} {row[6]:>3d} {row[7]:>4d} {row[8]:<10s} {row[9]:>5.3f}")

### Trischa Zorn — Most decorated Paralympic athlete (blind swimmer, 55 medals)

This result is especially interesting. Watch for visually impaired athletes from *different sports* appearing in the results — the embeddings capture the disability classification signal, connecting athletes across sport boundaries by shared experience.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            a2.name,
            a2.primary_sport,
            a2.games_type,
            a2.classification_code,
            a2.gold_count,
            a2.silver_count,
            a2.bronze_count,
            a2.total_medals,
            a2.first_games_year || '-' || a2.last_games_year AS career,
            ROUND((1 - (a1.embedding <=> a2.embedding))::numeric, 3) AS similarity
        FROM athletes a1
        CROSS JOIN athletes a2
        WHERE a1.name = 'Trischa Zorn'
          AND a2.athlete_id != a1.athlete_id
          AND a2.embedding IS NOT NULL
        ORDER BY a1.embedding <=> a2.embedding
        LIMIT 10
    """))

    print("🔍 Athletes most similar to Trischa Zorn:\n")
    print(f"   {'Name':<30s} {'Sport':<22s} {'Type':<10s} {'Class':<8s} {'🥇':>3s} {'🥈':>3s} {'🥉':>3s} {'Tot':>4s} {'Career':<10s} {'Sim':>5s}")
    print(f"   {'-'*30} {'-'*22} {'-'*10} {'-'*8} {'-'*3} {'-'*3} {'-'*3} {'-'*4} {'-'*10} {'-'*5}")
    for row in result:
        cls = row[3] if row[3] else ""
        print(f"   {row[0]:<30s} {row[1]:<22s} {row[2]:<10s} {cls:<8s} {row[4]:>3d} {row[5]:>3d} {row[6]:>3d} {row[7]:>4d} {row[8]:<10s} {row[9]:>5.3f}")

---
## Step 10: Filtered Similarity Search

Pure vector search is powerful, but real-world applications almost always combine it with relational filters. This is where AlloyDB shines over standalone vector databases — you can mix SQL `WHERE` clauses with vector similarity in a single query.

The pattern: use `ORDER BY embedding <=>` for similarity ranking, and standard SQL filters to narrow the scope. One query, one database.

In [ ]:
# Find Winter athletes with careers most similar to Michael Phelps
# Change the name or the WHERE filter and re-run!
search_athlete = "Michael Phelps"
filter_clause = "a2.games_season = 'Winter'"

with engine.connect() as conn:
    result = conn.execute(text(f"""
        SELECT
            a2.name,
            a2.primary_sport,
            a2.games_type,
            a2.games_season,
            a2.gold_count,
            a2.silver_count,
            a2.bronze_count,
            a2.total_medals,
            a2.first_games_year || '-' || a2.last_games_year AS career,
            ROUND((1 - (a1.embedding <=> a2.embedding))::numeric, 3) AS similarity
        FROM athletes a1
        CROSS JOIN athletes a2
        WHERE a1.name like :athlete
          AND a2.athlete_id != a1.athlete_id
          AND a2.embedding IS NOT NULL
          AND {filter_clause}
        ORDER BY a1.embedding <=> a2.embedding
        LIMIT 10
    """), {"athlete": search_athlete})

    print(f"🔍 Athletes matching filter '{filter_clause}' most similar to {search_athlete}:\n")
    print(f"   {'Name':<30s} {'Sport':<22s} {'Season':<8s} {'🥇':>3s} {'🥈':>3s} {'🥉':>3s} {'Tot':>4s} {'Career':<10s} {'Sim':>5s}")
    print(f"   {'-'*30} {'-'*22} {'-'*8} {'-'*3} {'-'*3} {'-'*3} {'-'*4} {'-'*10} {'-'*5}")
    for row in result:
        print(f"   {row[0]:<30s} {row[1]:<22s} {row[3]:<8s} {row[4]:>3d} {row[5]:>3d} {row[6]:>3d} {row[7]:>4d} {row[8]:<10s} {row[9]:>5.3f}")

---
## Step 11: Search by Natural Language Description

Here's what the agent will do in Task 5: instead of looking up a specific athlete's embedding, **generate an embedding from a text description on the fly** using AlloyDB's built-in `embedding()` function, then find the nearest athletes.

This means you can search by *meaning* — describe what you're looking for in plain English, and AlloyDB finds the closest matches.

In [ ]:
# Try different descriptions by changing this variable!
search_description = "dominant swimmer with many gold medals across multiple Olympics"

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            name,
            primary_sport,
            games_type,
            gold_count,
            silver_count,
            bronze_count,
            total_medals,
            first_games_year || '-' || last_games_year AS career,
            ROUND((1 - (embedding <=> embedding('gemini-embedding-001', :query)::vector))::numeric, 3) AS similarity
        FROM athletes
        WHERE embedding IS NOT NULL
        ORDER BY embedding <=> embedding('gemini-embedding-001', :query)::vector
        LIMIT 10
    """), {"query": search_description})

    print(f"🔍 Athletes matching: \"{search_description}\"\n")
    print(f"   {'Name':<30s} {'Sport':<22s} {'Type':<10s} {'🥇':>3s} {'🥈':>3s} {'🥉':>3s} {'Tot':>4s} {'Career':<10s} {'Sim':>5s}")
    print(f"   {'-'*30} {'-'*22} {'-'*10} {'-'*3} {'-'*3} {'-'*3} {'-'*4} {'-'*10} {'-'*5}")
    for row in result:
        print(f"   {row[0]:<30s} {row[1]:<22s} {row[2]:<10s} {row[3]:>3d} {row[4]:>3d} {row[5]:>3d} {row[6]:>4d} {row[7]:<10s} {row[8]:>5.3f}")

### Try other descriptions!

Change the `search_description` variable above and re-run the cell. Some ideas:

- `"Paralympic track and field athlete competing in wheelchair events"`
- `"Winter sport athlete with a long career spanning many Games"`
- `"Young gymnast who became a cultural icon"`
- `"Team sport athlete who won gold in basketball or volleyball"`

This is exactly the query pattern the agent will use in Task 5 — converting a user's natural language question into a vector and searching AlloyDB for the most relevant athletes.

---
## Step 12: Create the Results Table

While the `athletes` table captures career summaries, the `results` table stores **individual event outcomes** — every medal (and non-medal finish) for every athlete across every Games.

This enables queries like "what events did Simone Biles compete in?" or "show me all gold medals in gymnastics at the 2020 Games."

Using the same approach we used above with `athletes`, the cells below create the `results` table and load the data from the CSV into it.

In [ ]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS results"))
    conn.execute(text("""
        CREATE TABLE results (
            result_id           SERIAL PRIMARY KEY,
            athlete_id          VARCHAR(36) REFERENCES athletes(athlete_id),
            athlete_name        VARCHAR NOT NULL,
            games_year          INTEGER,
            games_season        VARCHAR,
            games_type          VARCHAR,
            sport               VARCHAR,
            discipline          VARCHAR,
            event               VARCHAR,
            medal               VARCHAR,
            classification_code VARCHAR
        )
    """))
    conn.commit()

print("✅ Results table created!")
print("   Columns: result_id (auto), athlete_id (FK), athlete_name, games_year,")
print("            games_season, games_type, sport, discipline, event, medal, classification_code")

In [ ]:
# Download the results CSV from the lab's GCS bucket
print("⏳ Downloading results data from Cloud Storage...")
df_results = pd.read_csv("gs://class-demo/team-usa/final/team_usa_results.csv")
print(f"   Downloaded {len(df_results):,} results with {len(df_results.columns)} columns")

# Select columns in table order (excluding result_id — Postgres generates that)
columns = [
    'athlete_id', 'athlete_name', 'games_year', 'games_season', 'games_type',
    'sport', 'discipline', 'event', 'medal', 'classification_code'
]
df_results_alloydb = df_results[columns].copy()

# Convert games_year from float to nullable int (NaN → empty string for CSV)
df_results_alloydb['games_year'] = df_results_alloydb['games_year'].apply(
    lambda x: '' if pd.isna(x) else str(int(x))
)

# Save without headers — GCS import maps columns positionally
local_results_path = "/tmp/results_for_alloydb.csv"
df_results_alloydb.to_csv(local_results_path, index=False, header=False)

print(f"✅ Prepared {len(df_results_alloydb):,} results ({len(columns)} columns) for import")
print(f"   Saved to {local_results_path}")


In [ ]:
# Upload the prepared CSV to the staging bucket
!gcloud storage cp {local_results_path} gs://{STAGING_BUCKET}/results_for_alloydb.csv --quiet

print(f"✅ Data staged to gs://{STAGING_BUCKET}/results_for_alloydb.csv")

In [ ]:
print("⏳ Importing results into AlloyDB (this may take 1-2 minutes)...")

!gcloud alloydb clusters import {CLUSTER_ID} \
    --project={PROJECT_ID} \
    --region={REGION} \
    --database={DATABASE_NAME} \
    --user={USER_EMAIL} \
    --csv \
    --table=results \
    --gcs-uri=gs://{STAGING_BUCKET}/results_for_alloydb.csv \
    --columns=athlete_id,athlete_name,games_year,games_season,games_type,sport,discipline,event,medal,classification_code

print("\n✅ Import complete!")

In [ ]:
with engine.connect() as conn:
    # Total count
    result = conn.execute(text("SELECT COUNT(*) FROM results"))
    total = result.scalar()

    # Medal breakdown
    result = conn.execute(text("""
        SELECT medal, COUNT(*)
        FROM results
        WHERE medal IS NOT NULL
        GROUP BY medal
        ORDER BY COUNT(*) DESC
    """))
    medals = result.fetchall()

    # Results with athlete FK match
    result = conn.execute(text("""
        SELECT COUNT(*) FROM results r
        JOIN athletes a ON r.athlete_id = a.athlete_id
    """))
    matched = result.scalar()

    print(f"✅ Results loaded: {total:,}")
    print(f"   Matched to athletes: {matched:,}")
    print(f"\n   Medal breakdown:")
    for medal, count in medals:
        print(f"   {medal}: {count:,}")

    # Sample some results
    print("\n📋 Sample results:")
    result = conn.execute(text("""
        SELECT athlete_name, games_year, sport, event, medal
        FROM results
        WHERE medal = 'Gold'
        ORDER BY games_year DESC
        LIMIT 8
    """))
    print(f"   {'Athlete':<25s} {'Year':>5s} {'Sport':<20s} {'Event':<30s} {'Medal'}")
    print(f"   {'-'*25} {'-'*5} {'-'*20} {'-'*30} {'-'*6}")
    for row in result:
        year = str(row[1]) if row[1] else 'N/A'
        print(f"   {row[0]:<25s} {year:>5s} {row[2]:<20s} {row[3]:<30s} {row[4]}")

---
## ✅ Task 4 Complete!

You've set up AlloyDB as the operational search layer for the Team USA platform:

- **Created** the `team_usa` database with vector extensions
- **Loaded** 12,000+ athletes with 3,072-dimension embeddings
- **Built** a ScaNN index for fast similarity search
- **Searched** by specific athlete (Simone Biles, Trischa Zorn)
- **Searched** by natural language description using `embedding()`
- **Loaded** 25,000+ event results with foreign key links to athletes

Your AlloyDB instance now has two tables working together:
- `athletes` — career summaries with vector embeddings for semantic search
- `results` — individual event outcomes for detailed competition queries

**What's next:** In Task 5, you'll build an AI agent that combines BigQuery (analytics, clustering) with AlloyDB (similarity search, event lookups) — choosing the right tool for each question automatically.